In [36]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/lending_club_2020_train_small_original.csv")

# predicted_y 칼럼에 값이 존재하는 row만 선택하여 df에 다시 저장
df = df[df['predicted_y'].notna()]

In [3]:
df.head(30)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,124989905,6000.0,6000.0,6000.000000,36 months,7.97%,187.94,A,A5,Teacher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,139665484,35000.0,35000.0,35000.000000,60 months,14.47%,822.95,C,C2,Product Specialist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,84253847,23200.0,23200.0,23200.000000,60 months,24.99%,680.82,E,E4,Production Foreman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,126296074,4500.0,4500.0,4500.000000,36 months,9.93%,145.06,B,B2,Office Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,148913312,15000.0,15000.0,15000.000000,36 months,16.40%,530.33,C,C4,Secretary,...,May-2020,Jul-2020,May-2020,2.0,0.0,ACTIVE,413.83000,11025.25,54.62,N
5,119708428,16000.0,16000.0,16000.000000,36 months,7.07%,494.55,A,A2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
6,109906493,4500.0,4500.0,4500.000000,36 months,10.42%,146.10,B,B3,Team Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
7,60286250,20000.0,20000.0,20000.000000,36 months,9.99%,645.25,B,B3,teacher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
8,151871973,19000.0,19000.0,18999.739941,60 months,17.97%,482.17,D,D1,Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
9,152325146,35000.0,35000.0,35000.000000,36 months,8.19%,1099.85,A,A4,Chief Financial Officer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


### 만기 36개월과 60개월을 나누기

In [37]:
# term 컬럼 값에 혹시 공백이 있다면 제거(필요할 경우)
df['term'] = df['term'].str.strip()  # 예: ' 36 months' -> '36 months'

# term이 '36 months'인 행만 추출하고 term 컬럼은 드롭
df_36 = df[df['term'] == '36 months'].drop('term', axis=1)

# term이 '60 months'인 행만 추출하고 term 컬럼은 드롭
df_60 = df[df['term'] == '60 months'].drop('term', axis=1)

statuses_to_keep = ['Fully Paid', 'Charged Off', 'Default']
df_36 = df_36[df_36['loan_status'].isin(statuses_to_keep)]
df_60 = df_60[df_60['loan_status'].isin(statuses_to_keep)]

In [38]:
df_36['loan_status'].value_counts()

loan_status
Fully Paid     944
Charged Off    216
Default          1
Name: count, dtype: int64

In [39]:
df_60['loan_status'].value_counts()

loan_status
Fully Paid     238
Charged Off     98
Name: count, dtype: int64

### 개별 만기에 대한 수익률 구하기

In [12]:
pip install numpy_financial

Note: you may need to restart the kernel to use updated packages.


In [40]:
import numpy_financial as npf
import pandas as pd

def calculate_actual_irr(row, n_months):
    """
    대출 상태에 따라 실제 IRR(연환산)을 계산합니다.
    
    - Fully Paid:
        현금흐름 = [-loan_amnt, installment, installment, ..., installment] (총 n_months회 지급)
    - Charged Off / Default:
        지급 횟수 p를 정기 상환금과 총 지급액으로 근사:
            p ≈ total_pymnt / installment
        현금흐름:
            초기: -loan_amnt
            중간: installment가 p-1회 지급
            마지막: installment와 recoveries(회수액)가 합산되어 지급
        
    매개변수:
        row: DataFrame의 한 행 (대출 정보)
        n_months: 대출 만기 (예: 36 또는 60)
    
    반환:
        연환산 IRR (float) 또는 계산 불가 시 None
    """
    loan_amnt = row['loan_amnt']
    status = row['loan_status']
    
    if status == 'Fully Paid':
        # Fully Paid: 예정된 만기 동안 매월 동일한 installment 지급
        cash_flows = [-loan_amnt] + [row['installment']] * n_months
        monthly_irr = npf.irr(cash_flows)
        if monthly_irr is None or not isinstance(monthly_irr, (int, float)):
            return None
        return (1 + monthly_irr)**12 - 1
    
    elif status in ['Charged Off', 'Default']:
        # 지급 횟수 p를 total_pymnt와 installment로 근사
        installment = row['installment']
        total_pymnt = row['total_pymnt']
        if installment and installment != 0:
            # p가 소수점이 나올 수 있으므로 반올림 후 정수화
            p = int(round(total_pymnt / installment))
            p = max(p, 1)  # 최소 1회 이상 지급되었음을 보장
        else:
            p = int(n_months / 2)  # fallback
        
        # 현금흐름 구성:
        # 초기 현금흐름: 대출금 상환 (-loan_amnt)
        # 이후 p-1회는 installment 지급, 마지막 회차에는 installment와 recoveries 추가
        if p == 1:
            # 지급 횟수가 1회인 경우
            last_cf = installment
            if 'recoveries' in row and not pd.isna(row['recoveries']):
                last_cf += row['recoveries']
            cash_flows = [-loan_amnt, last_cf]
        else:
            cash_flows = [-loan_amnt] + [installment] * (p - 1)
            last_payment = installment
            if 'recoveries' in row and not pd.isna(row['recoveries']):
                last_payment += row['recoveries']
            cash_flows.append(last_payment)
        
        monthly_irr = npf.irr(cash_flows)
        if monthly_irr is None or not isinstance(monthly_irr, (int, float)):
            return None
        return (1 + monthly_irr)**12 - 1
    
    else:
        # 그 외 상태는 처리하지 않음
        return None

# 예시 적용: df_36와 df_60에 대해 실제 IRR 계산 (만기 36, 60개월)
df_36['actual_irr'] = df_36.apply(lambda row: calculate_actual_irr(row, 36), axis=1)
df_60['actual_irr'] = df_60.apply(lambda row: calculate_actual_irr(row, 60), axis=1)

# 결과 확인 예시
print("36개월 대출자의 실제 IRR (연환산):")
print(df_36[['loan_amnt', 'installment', 'total_pymnt', 'recoveries', 'loan_status', 'actual_irr']].head(20))

print("\n60개월 대출자의 실제 IRR (연환산):")
print(df_60[['loan_amnt', 'installment', 'total_pymnt', 'recoveries', 'loan_status', 'actual_irr']].head(20))

36개월 대출자의 실제 IRR (연환산):
      loan_amnt  installment   total_pymnt  recoveries  loan_status  \
47      10000.0       340.18  12208.077967        0.00   Fully Paid   
123      5000.0       157.13   5113.365365        0.00   Fully Paid   
164      6000.0       196.18   6766.699041        0.00   Fully Paid   
238     15000.0       571.70   9128.990000        0.00  Charged Off   
269     10875.0       369.00  11461.701101        0.00   Fully Paid   
388     20000.0       684.24  21214.187225        0.00   Fully Paid   
680      6600.0       217.05   6631.099500        0.00   Fully Paid   
734      7000.0       210.81   7487.879097        0.00   Fully Paid   
756     37225.0      1169.77   2305.670000        0.00  Charged Off   
793     20000.0       743.17  12948.600000     1823.26  Charged Off   
916     15000.0       463.64  16667.710000        0.00   Fully Paid   
955     14000.0       458.74   2335.680000        0.00  Charged Off   
967      1500.0        47.06   1655.691701        0.0

### 각 df에 국채 금리 칼럼 추가하기

In [30]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [41]:
import datetime
import pandas as pd
import numpy as np
from pandas_datareader import data as web
from dateutil.relativedelta import relativedelta

# ※ 가정: df_36와 df_60는 이미 로드되어 있고, 'issue_d' 컬럼은 '%b-%Y' (예: "Dec-2017") 형태의 문자열 또는 datetime으로 되어 있음
# 만약 'issue_d'가 datetime이라면, 매핑을 위해 문자열 형태로 변환합니다.
df_36['issue_d_str'] = df_36['issue_d'].apply(lambda x: x.strftime('%b-%Y') if isinstance(x, pd.Timestamp) else x)
df_60['issue_d_str'] = df_60['issue_d'].apply(lambda x: x.strftime('%b-%Y') if isinstance(x, pd.Timestamp) else x)

##############################################
# 1. 36개월 대출자 – DGS3 (3년물) 일별 데이터 중 평균값 활용
##############################################
df36_clean = df_36.dropna(subset=['issue_d_str']).copy()
issue_d_unique_36 = df36_clean['issue_d_str'].unique()

results_3yr = []
for val in issue_d_unique_36:
    try:
        # 문자열을 datetime으로 변환 (예: "Dec-2017")
        dt = datetime.datetime.strptime(val, '%b-%Y')
        # 해당 월의 시작일과 말일 구하기
        start_date_dt = dt.replace(day=1)
        end_date_dt = (start_date_dt + relativedelta(months=1)) - datetime.timedelta(days=1)
        
        # FRED에서 DGS3 데이터 읽기 (일별)
        df_3yr = web.DataReader('DGS3', 'fred', start_date_dt, end_date_dt)
        if df_3yr.empty or df_3yr['DGS3'].dropna().empty:
            avg_val = np.nan
        else:
            # 해당 달의 평균 금리 (무위험수익률로 활용)
            avg_val = df_3yr['DGS3'].dropna().mean() / 100
        
        results_3yr.append({
            "issue_d": val,
            "start_date": start_date_dt.strftime('%Y-%m-%d'),
            "end_date": end_date_dt.strftime('%Y-%m-%d'),
            "risk_free_rate": avg_val
        })
    except Exception as e:
        results_3yr.append({
            "issue_d": val,
            "start_date": None,
            "end_date": None,
            "risk_free_rate": np.nan
        })

# 월별 3년물 통계 DataFrame 생성 및 출력
df_3yr_stats = pd.DataFrame(results_3yr)
print("===== 3년물 (FRED: DGS3) 월별 평균 금리 =====")
print(df_3yr_stats)

# issue_d 기준으로 risk_free_rate 매핑
mapping_3yr = dict(zip(df_3yr_stats['issue_d'], df_3yr_stats['risk_free_rate']))
df_36['risk_free_rate'] = df_36['issue_d_str'].map(mapping_3yr)

===== 3년물 (FRED: DGS3) 월별 평균 금리 =====
     issue_d  start_date    end_date  risk_free_rate
0   May-2016  2016-05-01  2016-05-31        0.009719
1   Feb-2019  2019-02-01  2019-02-28        0.024816
2   Nov-2017  2017-11-01  2017-11-30        0.018076
3   May-2018  2018-05-01  2018-05-31        0.026573
4   May-2017  2017-05-01  2017-05-31        0.014836
5   Mar-2018  2018-03-01  2018-03-31        0.024219
6   Nov-2018  2018-11-01  2018-11-30        0.029075
7   Aug-2016  2016-08-01  2016-08-31        0.008539
8   Oct-2019  2019-10-01  2019-10-31        0.015282
9   Mar-2017  2017-03-01  2017-03-31        0.015857
10  Aug-2017  2017-08-01  2017-08-31        0.014783
11  Sep-2018  2018-09-01  2018-09-30        0.028374
12  Jun-2018  2018-06-01  2018-06-30        0.026452
13  Apr-2017  2017-04-01  2017-04-30        0.014421
14  Mar-2016  2016-03-01  2016-03-31        0.010382
15  Apr-2016  2016-04-01  2016-04-30        0.009186
16  Oct-2017  2017-10-01  2017-10-31        0.016829
17  Jul-

In [42]:
##############################################
# 2. 60개월 대출자 – DGS5 (5년물) 일별 데이터 중 평균값 활용
##############################################
df60_clean = df_60.dropna(subset=['issue_d_str']).copy()
issue_d_unique_60 = df60_clean['issue_d_str'].unique()

results_5yr = []
for val in issue_d_unique_60:
    try:
        dt = datetime.datetime.strptime(val, '%b-%Y')
        start_date_dt = dt.replace(day=1)
        end_date_dt = (start_date_dt + relativedelta(months=1)) - datetime.timedelta(days=1)
        
        # FRED에서 DGS5 데이터 읽기 (일별)
        df_5yr = web.DataReader('DGS5', 'fred', start_date_dt, end_date_dt)
        if df_5yr.empty or df_5yr['DGS5'].dropna().empty:
            avg_val = np.nan
        else:
            # 해당 달의 평균 금리 (무위험수익률로 활용)
            avg_val = df_5yr['DGS5'].dropna().mean() / 100
        
        results_5yr.append({
            "issue_d": val,
            "start_date": start_date_dt.strftime('%Y-%m-%d'),
            "end_date": end_date_dt.strftime('%Y-%m-%d'),
            "risk_free_rate": avg_val
        })
    except Exception as e:
        results_5yr.append({
            "issue_d": val,
            "start_date": None,
            "end_date": None,
            "risk_free_rate": np.nan
        })

# 월별 5년물 통계 DataFrame 생성 및 출력
df_5yr_stats = pd.DataFrame(results_5yr)
print("===== 5년물 (FRED: DGS5) 월별 평균 금리 =====")
print(df_5yr_stats)

# issue_d 기준으로 risk_free_rate 매핑
mapping_5yr = dict(zip(df_5yr_stats['issue_d'], df_5yr_stats['risk_free_rate']))
df_60['risk_free_rate'] = df_60['issue_d_str'].map(mapping_5yr)

===== 5년물 (FRED: DGS5) 월별 평균 금리 =====
     issue_d  start_date    end_date  risk_free_rate
0   Mar-2016  2016-03-01  2016-03-31        0.013759
1   Mar-2017  2017-03-01  2017-03-31        0.020135
2   Jul-2016  2016-07-01  2016-07-31        0.010715
3   Jun-2017  2017-06-01  2017-06-30        0.017741
4   Aug-2016  2016-08-01  2016-08-31        0.011330
5   Jun-2019  2019-06-01  2019-06-30        0.018250
6   Jan-2020  2020-01-01  2020-01-31        0.015624
7   May-2017  2017-05-01  2017-05-31        0.018368
8   Nov-2016  2016-11-01  2016-11-30        0.015960
9   Feb-2018  2018-02-01  2018-02-28        0.025995
10  Nov-2017  2017-11-01  2017-11-30        0.020500
11  Feb-2016  2016-02-01  2016-02-29        0.012220
12  Jun-2016  2016-06-01  2016-06-30        0.011664
13  Mar-2019  2019-03-01  2019-03-31        0.023729
14  Aug-2017  2017-08-01  2017-08-31        0.017765
15  Sep-2017  2017-09-01  2017-09-30        0.017975
16  Sep-2018  2018-09-01  2018-09-30        0.028937
17  Mar-

### Sharpe ratio 구해보기

#### 회사 전체의 sharpe ratio와 수익률

In [43]:
import numpy as np
import pandas as pd

# 1. 36개월, 60개월 대출 데이터를 하나로 통합 (실제 IRR, 무위험수익률, loan_amnt 등 필요한 칼럼이 존재한다고 가정)
df_all = pd.concat([df_36, df_60], ignore_index=True)

# 2. 각 대출별 초과수익률 계산: actual_irr - risk_free_rate
df_all['excess_return'] = df_all['actual_irr'] - df_all['risk_free_rate']

# 3. 가중 표준편차 계산 함수 정의 (가중치는 loan_amnt 사용)
def weighted_std(values, weights):
    weighted_mean = np.sum(values * weights) / np.sum(weights)
    variance = np.sum(weights * (values - weighted_mean)**2) / np.sum(weights)
    return np.sqrt(variance)

# 4. 가중 평균 초과수익률 계산
weighted_avg_excess = np.sum(df_all['loan_amnt'] * df_all['excess_return']) / np.sum(df_all['loan_amnt'])

# 5. 가중 표준편차 계산
w_std = weighted_std(df_all['excess_return'], df_all['loan_amnt'])

# 6. Sharpe Ratio 계산
sharpe_ratio = weighted_avg_excess / w_std

print("Company Sharpe Ratio:", sharpe_ratio)

# df_all은 앞서 df_36와 df_60를 통합한 DataFrame입니다.
company_return = (df_all['loan_amnt'] * df_all['actual_irr']).sum() / df_all['loan_amnt'].sum()

print("Company Weighted Average Actual IRR: {:.4f}".format(company_return))

Company Sharpe Ratio: -0.06457479569660642
Company Weighted Average Actual IRR: -0.0041


#### 대출 만기별 수익률과 sharpe ratio 구하기

In [44]:
import numpy as np
import pandas as pd

# 1. 36개월 대출자 전체의 가중 평균 실제 IRR 계산
weighted_avg_actual_irr_36 = (df_36['loan_amnt'] * df_36['actual_irr']).sum() / df_36['loan_amnt'].sum()
print("36개월 대출자 Weighted Average Actual IRR: {:.4f}".format(weighted_avg_actual_irr_36))

# 2. 60개월 대출자 전체의 가중 평균 실제 IRR 계산
weighted_avg_actual_irr_60 = (df_60['loan_amnt'] * df_60['actual_irr']).sum() / df_60['loan_amnt'].sum()
print("60개월 대출자 Weighted Average Actual IRR: {:.4f}".format(weighted_avg_actual_irr_60))

# 3. 초과수익률(실제 IRR - 무위험수익률) 계산
df_36['excess_return'] = df_36['actual_irr'] - df_36['risk_free_rate']
df_60['excess_return'] = df_60['actual_irr'] - df_60['risk_free_rate']

# 4. 가중 표준편차 계산 함수 (대출금액을 가중치로 사용)
def weighted_std(values, weights):
    weighted_mean = np.sum(values * weights) / np.sum(weights)
    variance = np.sum(weights * (values - weighted_mean)**2) / np.sum(weights)
    return np.sqrt(variance)

# 5. 36개월 대출자 초과수익률의 가중 평균 및 가중 표준편차 계산
weighted_avg_excess_36 = (df_36['loan_amnt'] * df_36['excess_return']).sum() / df_36['loan_amnt'].sum()
w_std_excess_36 = weighted_std(df_36['excess_return'], df_36['loan_amnt'])
sharpe_ratio_36 = weighted_avg_excess_36 / w_std_excess_36

# 6. 60개월 대출자 초과수익률의 가중 평균 및 가중 표준편차 계산
weighted_avg_excess_60 = (df_60['loan_amnt'] * df_60['excess_return']).sum() / df_60['loan_amnt'].sum()
w_std_excess_60 = weighted_std(df_60['excess_return'], df_60['loan_amnt'])
sharpe_ratio_60 = weighted_avg_excess_60 / w_std_excess_60

print("36개월 대출자 Sharpe Ratio: {:.4f}".format(sharpe_ratio_36))
print("60개월 대출자 Sharpe Ratio: {:.4f}".format(sharpe_ratio_60))

36개월 대출자 Weighted Average Actual IRR: 0.0151
60개월 대출자 Weighted Average Actual IRR: -0.0457
36개월 대출자 Sharpe Ratio: -0.0015
60개월 대출자 Sharpe Ratio: -0.1652


### (향후 적용) 부도예측 모델 시행 이후 수익률과 sharpe ratio 구하기

In [45]:
import numpy as np
import pandas as pd

# --- 1. 전체 포트폴리오 (df_all)는 이미 df_36와 df_60를 합친 상태라고 가정 ---
# 예) df_all = pd.concat([df_36, df_60], ignore_index=True)

# --- 2. 우리 모델의 투자 결정에 따른 수익률 할당 ---
# 만약 pred_default == 1이면 실제 대출 투자 -> actual_irr 사용
# 만약 pred_default == 0이면 투자하지 않음 -> 해당 대출 실행 시 무위험수익률(risk_free_rate) 사용
df_all['model_return'] = df_all.apply(
    lambda row: row['actual_irr'] if row['predicted_y'] == 1 else row['risk_free_rate'],
    axis=1
)

# --- 3. 모델 포트폴리오의 가중 평균 수익률 계산 ---
weighted_avg_model_return = (df_all['loan_amnt'] * df_all['model_return']).sum() / df_all['loan_amnt'].sum()
print("모델 포트폴리오 Weighted Average Return: {:.4f}".format(weighted_avg_model_return))

# --- 4. 초과수익률 계산 --- 
# 각 대출별로 (model_return - risk_free_rate)를 계산합니다.
# (즉, pred_default==1 인 경우에는 actual_irr - risk_free_rate, pred_default==0 인 경우에는 0)
df_all['excess_return_model'] = df_all['model_return'] - df_all['risk_free_rate']

# --- 5. 가중 표준편차 계산 함수 (대출금액을 가중치로 사용) ---
def weighted_std(values, weights):
    weighted_mean = np.sum(values * weights) / np.sum(weights)
    variance = np.sum(weights * (values - weighted_mean)**2) / np.sum(weights)
    return np.sqrt(variance)

# 모델 포트폴리오의 가중 평균 초과수익률
weighted_avg_excess_model = (df_all['loan_amnt'] * df_all['excess_return_model']).sum() / df_all['loan_amnt'].sum()

# 모델 포트폴리오의 가중 표준편차 계산
w_std_excess_model = weighted_std(df_all['excess_return_model'], df_all['loan_amnt'])

# --- 6. 모델 포트폴리오 Sharpe Ratio 계산 ---
# (초과수익률의 가중 평균 / 가중 표준편차)
sharpe_ratio_model = weighted_avg_excess_model / w_std_excess_model

print("모델 포트폴리오 Sharpe Ratio: {:.4f}".format(sharpe_ratio_model))

모델 포트폴리오 Weighted Average Return: 0.0854
모델 포트폴리오 Sharpe Ratio: 0.4166


In [46]:
import numpy as np
import pandas as pd

# --- 1. 기존 렌딩클럽 전략 (전체 포트폴리오) ---
# df_all: df_all = pd.concat([df_36, df_60], ignore_index=True) 로 전체 포트폴리오가 구성되어 있다고 가정

# 각 대출의 초과수익률 계산 (actual_irr - risk_free_rate)
df_all['excess_return'] = df_all['actual_irr'] - df_all['risk_free_rate']

# 가중 평균 실제 IRR 계산 (대출금액을 가중치로 사용)
weighted_avg_actual_irr_orig = (df_all['loan_amnt'] * df_all['actual_irr']).sum() / df_all['loan_amnt'].sum()

# 가중 평균 초과수익률 계산
weighted_avg_excess_orig = (df_all['loan_amnt'] * df_all['excess_return']).sum() / df_all['loan_amnt'].sum()

# 가중 표준편차 계산 함수 (대출금액을 가중치로 사용)
def weighted_std(values, weights):
    weighted_mean = np.sum(values * weights) / np.sum(weights)
    variance = np.sum(weights * (values - weighted_mean)**2) / np.sum(weights)
    return np.sqrt(variance)

w_std_excess_orig = weighted_std(df_all['excess_return'], df_all['loan_amnt'])

# 기존 전략의 Sharpe Ratio 계산
sharpe_ratio_orig = weighted_avg_excess_orig / w_std_excess_orig

# --- 2. 우리 모델 전략 ---
# 우리 모델에서는 부도예측 모형 결과가 이진 변수일 때, 
# pred_default == 1이면 해당 대출에 투자(실제 IRR 적용), 
# pred_default == 0이면 투자하지 않고 무위험수익률(risk_free_rate)을 적용한다고 가정합니다.
df_all['model_return'] = df_all.apply(
    lambda row: row['actual_irr'] if row['predicted_y'] == 1 else row['risk_free_rate'],
    axis=1
)

# 우리 모델 전략의 가중 평균 수익률 계산
weighted_avg_model_return = (df_all['loan_amnt'] * df_all['model_return']).sum() / df_all['loan_amnt'].sum()

# 우리 모델 전략의 초과수익률 계산 (model_return - risk_free_rate)
df_all['excess_return_model'] = df_all['model_return'] - df_all['risk_free_rate']
weighted_avg_excess_model = (df_all['loan_amnt'] * df_all['excess_return_model']).sum() / df_all['loan_amnt'].sum()
w_std_excess_model = weighted_std(df_all['excess_return_model'], df_all['loan_amnt'])
sharpe_ratio_model = weighted_avg_excess_model / w_std_excess_model

# --- 3. 비교 출력 ---
print("== 기존 렌딩클럽 전략 ==")
print("Weighted Average Actual IRR: {:.4f}".format(weighted_avg_actual_irr_orig))
print("Sharpe Ratio: {:.4f}".format(sharpe_ratio_orig))

print("\n== 우리 모델 ==")
print("Weighted Average Return (model): {:.4f}".format(weighted_avg_model_return))
print("Sharpe Ratio (model): {:.4f}".format(sharpe_ratio_model))

# 옵션: 테이블 형식으로 비교 출력
comparison_df = pd.DataFrame({
    "Strategy": ["Original", "Our Model"],
    "Weighted Average Return": [weighted_avg_actual_irr_orig, weighted_avg_model_return],
    "Sharpe Ratio": [sharpe_ratio_orig, sharpe_ratio_model]
})
print("\n--- 전략 비교 ---")
print(comparison_df)

== 기존 렌딩클럽 전략 ==
Weighted Average Actual IRR: -0.0041
Sharpe Ratio: -0.0646

== 우리 모델 ==
Weighted Average Return (model): 0.0854
Sharpe Ratio (model): 0.4166

--- 전략 비교 ---
    Strategy  Weighted Average Return  Sharpe Ratio
0   Original                -0.004080     -0.064575
1  Our Model                 0.085408      0.416647
